### 🧠 Síntesis de Respuestas desde Múltiples Fuentes
✅ ¿Qué es?

La síntesis de respuestas desde múltiples fuentes es el proceso donde un agente de IA recopila información de diferentes herramientas de recuperación o bases de conocimiento, y fusiona esa información en una única respuesta coherente y contextualmente rica.

Esta es una capacidad fundamental en RAG Agéntico, donde el sistema es más que un simple recuperador — planifica, recupera, y luego sintetiza una respuesta que se nutre de múltiples fuentes.


🎯 Por qué es necesario
La mayoría de las consultas del mundo real son:
- Multifacéticas (requieren múltiples tipos de información)
- Ambiguas o incompletas (necesitan refinamiento)
- Abiertas (no se mapean a un solo documento o fuente)

🔍 Esto hace que recuperar desde una sola base de datos vectorial sea insuficiente.

En su lugar, queremos un agente que pueda:

- Decidir qué obtener y de dónde (planificación de recuperación)
- Recuperar contenido de múltiples herramientas (ej., Wikipedia, PDFs, APIs, SQL)
- Evaluar y fusionar ese contexto
- Producir una única respuesta similar a la humana

In [ ]:
# Importación del módulo os para interactuar con el sistema operativo (variables de entorno, rutas, etc.)
import os

# Importación de List desde typing para definir tipos de datos (listas tipadas)
from typing import List

# Importación de BaseModel desde pydantic para crear modelos de datos con validación automática
from pydantic import BaseModel

# Importación de init_chat_model para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# Importación de OpenAIEmbeddings para generar embeddings (representaciones vectoriales) usando la API de OpenAI
from langchain_openai import OpenAIEmbeddings

# Importación de Document, la clase base de LangChain para representar documentos con contenido y metadatos
from langchain.schema import Document

# Importación de FAISS, una biblioteca de Facebook para búsqueda eficiente de similitud en vectores
from langchain.vectorstores import FAISS

# Importación de TextLoader para cargar archivos de texto plano (.txt) como documentos
from langchain_community.document_loaders import TextLoader

# Importación de YoutubeLoader para cargar transcripciones de videos de YouTube
from langchain_community.document_loaders.youtube import YoutubeLoader

# Importación de ArxivLoader para buscar y cargar papers académicos desde ArXiv
from langchain_community.document_loaders import ArxivLoader

# Importación de RecursiveCharacterTextSplitter para dividir textos largos en chunks (fragmentos) de manera recursiva
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Importación de WikipediaQueryRun para ejecutar búsquedas en Wikipedia
from langchain.tools import WikipediaQueryRun

# Importación de WikipediaAPIWrapper, el wrapper que encapsula las llamadas a la API de Wikipedia
from langchain.utilities import WikipediaAPIWrapper

# Importación de StateGraph para crear grafos de estado (flujos de trabajo con nodos y aristas)
# END es un marcador especial que indica el final del grafo
from langgraph.graph import StateGraph, END

In [ ]:
# Importación del módulo os para acceder a variables de entorno
import os

# Importación de init_chat_model para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# Importación de load_dotenv para cargar variables de entorno desde un archivo .env
from dotenv import load_dotenv

# Configuración de la variable de entorno OPENAI_API_KEY con el valor obtenido del archivo .env
# Esto permite autenticarse con la API de OpenAI
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# Inicialización del modelo de lenguaje usando GPT-4o-mini de OpenAI
# Se usa la versión 'mini' que es más económica y rápida, ideal para síntesis de información
llm=init_chat_model("openai:gpt-4o-mini")

In [ ]:
# Función para cargar y crear un retriever desde un archivo de texto
def load_text_retriever(file_path):
    """
    Carga un archivo de texto, lo divide en chunks y crea un retriever vectorial.
    Esto permite buscar información relevante en documentos internos.
    """
    # Carga del archivo de texto usando TextLoader
    # encoding="utf-8" asegura la correcta lectura de caracteres especiales y acentos
    docs = TextLoader(file_path, encoding="utf-8").load()
    
    # Creación de un splitter (divisor) para fragmentar los documentos largos
    # chunk_size=500: cada fragmento tendrá aproximadamente 500 caracteres
    # chunk_overlap=50: habrá una superposición de 50 caracteres entre fragmentos consecutivos
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    
    # División de los documentos en chunks más pequeños
    chunks = splitter.split_documents(docs)
    
    # Creación de un vector store usando FAISS con los chunks y embeddings de OpenAI
    vs = FAISS.from_documents(chunks, OpenAIEmbeddings())
    
    # Retorna el vector store convertido en retriever para búsquedas
    return vs.as_retriever()

# Función para cargar un retriever simulado de transcripciones de YouTube
def load_youtube_retriever():
    """
    En este ejemplo, crea un retriever simulado con contenido de YouTube.
    En producción, esto usaría YoutubeLoader con URLs reales de videos.
    """
    # Contenido simulado de una transcripción de YouTube sobre sistemas agénticos de IA
    # Este texto representa lo que se obtendría de un video real
    content = """
    Este video explica cómo los sistemas de IA agénticos dependen de bucles de retroalimentación, memoria y uso de herramientas.
    Los compara con LLMs tradicionales basados en pipelines. Se enfatizan el razonamiento temporal y las tareas autónomas.
    """
    # Creación de un objeto Document con el contenido simulado y metadatos de origen
    doc = Document(page_content=content, metadata={"source": "youtube"})
    
    # Creación de un vector store con el documento simulado
    vectorstore = FAISS.from_documents([doc], OpenAIEmbeddings())
    
    # Retorna el vector store convertido en retriever
    return vectorstore.as_retriever()



# Función para buscar información en Wikipedia
def wikipedia_search(query: str) -> str:
    """
    Realiza una búsqueda en Wikipedia y retorna el contenido relevante.
    Útil para obtener información general y contextual sobre temas amplios.
    """
    # Imprime un mensaje indicando que se está buscando en Wikipedia
    print("🌐 Buscando en Wikipedia...")
    
    # Ejecuta la búsqueda usando WikipediaQueryRun con el wrapper de la API
    # WikipediaAPIWrapper() maneja las llamadas HTTP a la API de Wikipedia
    # (query) invoca la búsqueda con la consulta proporcionada
    return WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())(query)

# Función para buscar papers académicos en ArXiv
def arxiv_search(query: str) -> str:
    """
    Busca y carga papers académicos desde ArXiv relacionados con la consulta.
    Ideal para obtener información científica actualizada y de investigación.
    """
    # Imprime un mensaje indicando que se está buscando en ArXiv
    print("📄 Buscando en ArXiv...")
    
    # Usa ArxivLoader para buscar papers relacionados con la consulta
    # .load() descarga y procesa los papers encontrados
    results = ArxivLoader(query).load()
    
    # Concatena el contenido de los primeros 2 papers ([:2]) separados por doble salto de línea
    # Si no hay resultados, retorna un mensaje indicando que no se encontraron papers
    # 'or' actúa como operador de respaldo: si el string está vacío, retorna el mensaje
    return "\n\n".join(doc.page_content for doc in results[:2]) or "No se encontraron papers relevantes."

In [ ]:
# Creación de los retrievers que se usarán para búsqueda de información

# text_retriever: retriever para documentos internos de texto
# Carga y procesa el archivo "internal_docs.txt" para búsquedas vectoriales
text_retriever = load_text_retriever("internal_docs.txt")

# youtube_retriever: retriever para contenido de YouTube (en este caso, simulado)
# En producción, esto cargaría transcripciones reales de videos de YouTube
youtube_retriever = load_youtube_retriever()

In [ ]:
### Estado del grafo para RAG multi-fuente

# Clase que define el estado del flujo de trabajo para síntesis de múltiples fuentes
# Hereda de BaseModel (Pydantic) para validación automática de tipos y datos
class MultiSourceRAGState(BaseModel):
    # question: la pregunta original del usuario (tipo string, campo obligatorio)
    question: str
    
    # text_docs: lista de documentos recuperados de archivos de texto internos
    # Por defecto es una lista vacía []
    # Contiene fragmentos relevantes de los documentos internos
    text_docs: List[Document] = []
    
    # yt_docs: lista de documentos recuperados de transcripciones de YouTube
    # Por defecto es una lista vacía []
    # Contiene información de videos relacionados con la consulta
    yt_docs: List[Document] = []
    
    # wiki_context: contexto/información recuperada de Wikipedia
    # Por defecto es una cadena vacía ""
    # Contiene el texto completo retornado por la búsqueda en Wikipedia
    wiki_context: str = ""
    
    # arxiv_context: contexto/información recuperada de papers académicos en ArXiv
    # Por defecto es una cadena vacía ""
    # Contiene fragmentos de papers científicos relevantes
    arxiv_context: str = ""
    
    # final_answer: la respuesta final sintetizada que combina información de todas las fuentes
    # Por defecto es una cadena vacía ""
    # Esta es la respuesta coherente y completa que se presenta al usuario
    final_answer: str = ""

In [ ]:
### Nodos de Recuperación - cada uno obtiene información de una fuente diferente

# Nodo 1: Recuperar documentos de texto internos
def retrieve_text(state: MultiSourceRAGState) -> MultiSourceRAGState:
    """
    Recupera documentos relevantes de archivos de texto internos.
    Primera fuente de información: documentación interna de la empresa.
    """
    # Invoca el retriever de texto con la pregunta del usuario
    # Busca fragmentos similares semánticamente en los documentos internos
    docs = text_retriever.invoke(state.question)
    
    # Retorna una copia actualizada del estado con los documentos de texto
    return state.model_copy(update={"text_docs": docs})

# Nodo 2: Recuperar contenido de YouTube
def retrieve_yt(state: MultiSourceRAGState) -> MultiSourceRAGState:
    """
    Recupera información de transcripciones de videos de YouTube.
    Segunda fuente: contenido multimedia/educativo de YouTube.
    """
    # Invoca el retriever de YouTube con la pregunta del usuario
    # Busca contenido relevante en las transcripciones de video
    docs = youtube_retriever.invoke(state.question)
    
    # Retorna una copia actualizada del estado con los documentos de YouTube
    return state.model_copy(update={"yt_docs": docs})

# Nodo 3: Recuperar información de Wikipedia
def retrieve_wikipedia(state: MultiSourceRAGState) -> MultiSourceRAGState:
    """
    Busca y recupera información general de Wikipedia.
    Tercera fuente: conocimiento enciclopédico público.
    """
    # Llama a la función de búsqueda de Wikipedia con la pregunta del usuario
    # Obtiene artículos y contenido relevante de la enciclopedia
    result = wikipedia_search(state.question)
    
    # Retorna una copia actualizada del estado con el contexto de Wikipedia
    return state.model_copy(update={"wiki_context": result})

# Nodo 4: Recuperar papers de ArXiv
def retrieve_arxiv(state: MultiSourceRAGState) -> MultiSourceRAGState:
    """
    Busca y recupera papers académicos de ArXiv.
    Cuarta fuente: investigación científica y académica actualizada.
    """
    # Llama a la función de búsqueda de ArXiv con la pregunta del usuario
    # Obtiene papers relevantes de investigación científica
    result = arxiv_search(state.question)
    
    # Retorna una copia actualizada del estado con el contexto de ArXiv
    return state.model_copy(update={"arxiv_context": result})

In [ ]:
## Nodo de Síntesis - combina toda la información en una respuesta coherente

def synthesize_answer(state: MultiSourceRAGState) -> MultiSourceRAGState:
    """
    Función central que sintetiza información de todas las fuentes en una respuesta unificada.
    Combina: documentos internos + YouTube + Wikipedia + ArXiv.
    """
    
    # Inicialización de una cadena vacía que contendrá todo el contexto combinado
    context = ""

    # Agregado de contexto de documentos internos
    # Se añade un encabezado "[Documentos Internos]" para identificar la fuente
    # Se concatena el contenido de cada documento separado por saltos de línea
    context += "\n\n[Documentos Internos]\n" + "\n".join([doc.page_content for doc in state.text_docs])
    
    # Agregado de contexto de transcripciones de YouTube
    # Se añade un encabezado "[Transcripción de YouTube]" para identificar la fuente
    context += "\n\n[Transcripción de YouTube]\n" + "\n".join([doc.page_content for doc in state.yt_docs])
    
    # Agregado de contexto de Wikipedia
    # Se añade un encabezado "[Wikipedia]" seguido del contenido recuperado
    context += "\n\n[Wikipedia]\n" + state.wiki_context
    
    # Agregado de contexto de ArXiv
    # Se añade un encabezado "[ArXiv]" seguido de los papers académicos
    context += "\n\n[ArXiv]\n" + state.arxiv_context

    # Construcción del prompt para el LLM
    # Se le pide explícitamente que sintetice la información de múltiples fuentes
    # El prompt incluye la pregunta y todo el contexto organizado por fuentes
    prompt = f"""Has recuperado contexto relevante de múltiples fuentes. Ahora sintetiza una respuesta completa y coherente.

Pregunta: {state.question}

Contexto:
{context}

Respuesta Final:"""

    # Invocación del LLM con el prompt completo
    # El LLM analiza toda la información de las cuatro fuentes y genera una respuesta sintetizada
    # .content.strip() extrae el texto de la respuesta y elimina espacios en blanco
    answer = llm.invoke(prompt).content.strip()
    
    # Retorna el estado actualizado con la respuesta final sintetizada
    return state.model_copy(update={"final_answer": answer})

In [ ]:
# Creación del constructor del grafo de estado usando la clase MultiSourceRAGState
# Este grafo implementa un flujo secuencial de recuperación desde múltiples fuentes
builder = StateGraph(MultiSourceRAGState)

# Agregado de nodos al grafo - cada uno representa una fuente de información

# Nodo para recuperar documentos de texto internos
builder.add_node("retrieve_text", retrieve_text)

# Nodo para recuperar transcripciones de YouTube
builder.add_node("retrieve_yt", retrieve_yt)

# Nodo para buscar información en Wikipedia
builder.add_node("retrieve_wiki", retrieve_wikipedia)

# Nodo para buscar papers académicos en ArXiv
builder.add_node("retrieve_arxiv", retrieve_arxiv)

# Nodo para sintetizar toda la información en una respuesta coherente
builder.add_node("synthesize", synthesize_answer)

# Establecer el punto de entrada del grafo (primer nodo a ejecutar)
# El flujo comienza recuperando documentos de texto internos
builder.set_entry_point("retrieve_text")

# Definición de aristas (edges) - flujo secuencial de recuperación

# Paso 1: Después de recuperar texto, ir a YouTube
builder.add_edge("retrieve_text", "retrieve_yt")

# Paso 2: Después de YouTube, ir a Wikipedia
builder.add_edge("retrieve_yt", "retrieve_wiki")

# Paso 3: Después de Wikipedia, ir a ArXiv
builder.add_edge("retrieve_wiki", "retrieve_arxiv")

# Paso 4: Después de ArXiv (todas las fuentes recuperadas), sintetizar la respuesta
builder.add_edge("retrieve_arxiv", "synthesize")

# Paso 5: Después de sintetizar, terminar el flujo
builder.add_edge("synthesize", END)

# Compilación del grafo para hacerlo ejecutable
# .compile() valida la estructura y optimiza el grafo final
graph = builder.compile()

# Mostrar el grafo compilado (en Jupyter esto puede renderizar una visualización)
graph

In [ ]:
# Definición de la pregunta del usuario
# Esta es una pregunta compleja que requiere información de múltiples fuentes:
# - Documentos internos sobre implementaciones específicas
# - YouTube para explicaciones conceptuales
# - Wikipedia para definiciones generales
# - ArXiv para investigación reciente
question = "¿Qué son los agentes transformers y cómo están evolucionando en la investigación reciente?"

# Creación del estado inicial con solo la pregunta del usuario
# Los demás campos (text_docs, yt_docs, wiki_context, arxiv_context, final_answer) usarán sus valores por defecto
state = MultiSourceRAGState(question=question)

# Invocación del grafo con el estado inicial
# .invoke() ejecuta todo el flujo secuencial:
# 1. Recupera de documentos internos
# 2. Recupera de YouTube
# 3. Recupera de Wikipedia
# 4. Recupera de ArXiv
# 5. Sintetiza toda la información en una respuesta coherente
result = graph.invoke(state)

# Impresión de la respuesta final sintetizada
# Esta respuesta combina información de las cuatro fuentes diferentes
# proporcionando una visión completa y multidimensional del tema
print("✅ Respuesta Final:\n")
print(result["final_answer"])


In [ ]:
# Mostrar el estado final completo para inspección y depuración
# Esto incluye todos los campos:
# - question: la pregunta original
# - text_docs: documentos recuperados de archivos internos
# - yt_docs: documentos recuperados de YouTube
# - wiki_context: contexto de Wikipedia
# - arxiv_context: contexto de ArXiv
# - final_answer: respuesta sintetizada final
# Útil para ver exactamente qué información se recuperó de cada fuente
result